<a href="https://colab.research.google.com/github/miratcan/BavulaNeKoysam/blob/master/pipeline_opensource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<cell_type>markdown</cell_type># linkinthe.video - Open Source Pipeline

**Modeller:**
- Whisper - Transcription (fallback)
- Qwen2 7B - Ürün çıkarma
- LLaVA 1.6 - Frame analizi

**Akış:**
1. Transcript al (YouTube altyazısı veya Whisper)
2. LLM ile ürün çıkar → found[] + lost[]
3. Lost varsa → Video indir → Frame çıkar → Vision ile tanı

**Lazy Evaluation:** Video sadece gerektiğinde indirilir.

---

<cell_type>markdown</cell_type>## 0. Kurulum

**ÖNEMLİ:** Runtime → Change runtime type → T4 GPU seç!

In [1]:
# GPU kontrol
!nvidia-smi

import torch
if torch.cuda.is_available():
    print(f"\n✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU YOK! Runtime → Change runtime type → T4 GPU seç!")

Fri Nov 28 12:04:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Paketleri kur
!pip install -q youtube-transcript-api openai-whisper
!pip install -q transformers accelerate bitsandbytes sentencepiece protobuf

✅ Kurulum tamamlandı!


In [ ]:
# Imports

import os
import json
import time
import requests

In [3]:
# Prepare working directory.
WORK_DIR = "/content/linkinthe_test"
os.makedirs(WORK_DIR, exist_ok=True)

In [4]:

# We are testing for only one video now, there will be multiple future.
VIDEO_URL = "https://www.youtube.com/watch?v=WlgjElhWD-U"

# State
local_video_path = "/content/drive/MyDrive/sample.webm"
transcript_segments = None  # timestamp'li
transcript_text = None

print(f"Video ID: {VIDEO_ID}")
print(f"Video URL: {VIDEO_URL}")

Video ID: WlgjElhWD-U
Video URL: https://www.youtube.com/watch?v=WlgjElhWD-U


## 1. Transcript Al

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi

# Önce YouTube altyazısını dene
try:
    transcript_segments = YouTubeTranscriptApi.get_transcript(VIDEO_ID, languages=['tr', 'en'])
    transcript_text = " ".join([s['text'] for s in transcript_segments])
    print(f"✅ YouTube altyazısı bulundu! ({len(transcript_text)} karakter)")
    print(f"   {len(transcript_segments)} segment (timestamp'li)")
    print("\n" + "="*60)
    print(transcript_text[:1000])
    print("="*60)
except Exception as e:
    print(f"❌ YouTube altyazısı yok: {e}")
    print("→ Video indirip Whisper kullanacağız...")

❌ YouTube altyazısı yok: type object 'YouTubeTranscriptApi' has no attribute 'get_transcript'
→ Video indirip Whisper kullanacağız...


## 2. Video (sadece altyazı yoksa)

YouTube altyazısı varsa bu adımı **ATLA**.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Video dosyası - Drive'a koy veya Colab'a yükle

if os.path.exists(local_video_path):
    size_mb = os.path.getsize(local_video_path) / (1024 * 1024)
    print(f"✅ Video mevcut: {local_video_path} ({size_mb:.1f} MB)")
else:
    print(f"❌ Video bulunamadı: {local_video_path}")
    print("   Drive'dan kopyala veya yükle")

✅ Video mevcut: /content/drive/MyDrive/sample.webm (107.2 MB)


In [27]:
import re

def format_ts(seconds):
    m, s = divmod(int(seconds), 60)
    return f"[{m}:{s:02d}]"

def parse_srt(srt_path):
    """SRT dosyasını YouTube API formatına çevir."""
    with open(srt_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # SRT pattern: index, timestamp, text
    pattern = r'(\d+)\n(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\n(.+?)(?=\n\n|\Z)'
    matches = re.findall(pattern, content, re.DOTALL)

    segments = []
    for idx, start, end, text in matches:
        # "00:01:23,456" -> saniye
        h, m, s = start.split(':')
        s, ms = s.split(',')
        start_sec = int(h)*3600 + int(m)*60 + int(s) + int(ms)/1000

        segments.append({
            'text': text.replace('\n', ' ').strip(),
            'start': start_sec
        })

    return segments

# Kullanım
transcript_segments = parse_srt(f"{WORK_DIR}/sample.srt")
transcript_with_ts = "\n".join([
    f"{format_ts(s['start'])} {s['text']}"
    for s in transcript_segments
])

print(f"✅ {len(transcript_segments)} segment yüklendi")
print(transcript_with_ts[:2000])


✅ 1173 segment yüklendi
[0:00] did you know that Amazon sells millions
[0:02] of items every single day that's right
[0:04] people all over the world are constantly
[0:07] buying some of the most random things
[0:09] from insane viral gadgets to ancient
[0:11] dinosaur fossils and even an actual
[0:14] house basically anything you can think
[0:16] of Amazon's got it but what happens with
[0:18] all the items that get returned some get
[0:21] tossed some get resold but the lucky
[0:23] ones they get turned into mystery boxes
[0:26] these boxes then get posted online and
[0:28] sold to people just like me but just
[0:30] like a box of chocolates you never know
[0:32] what you're going to get which is why
[0:33] today I'll be opening 10 Amazon returns
[0:36] mystery boxes here's how it's going to
[0:38] work my cousin Jessica and I will be
[0:39] working together in order to unbox and
[0:42] guess the value of 10 mystery boxes we
[0:44] will then place them on a price that we
[0:46] think

In [9]:
import whisper

def format_ts(seconds):
    m, s = divmod(int(seconds), 60)
    return f"[{m}:{s:02d}]"

audio_path = f"{WORK_DIR}/audio.wav"
!ffmpeg -i "{local_video_path}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{audio_path}" -y -loglevel error
print(f"✅ Ses çıkarıldı")

print("Whisper yükleniyor...")
model = whisper.load_model("medium")

print("Transkript alınıyor...")
result = model.transcribe(audio_path)

# YouTube API formatına çevir
transcript_segments = [
    {'text': seg['text'].strip(), 'start': seg['start']}
    for seg in result['segments']
]

# Timestamp'li format
transcript_with_ts = "\n".join([
    f"{format_ts(s['start'])} {s['text']}"
    for s in transcript_segments
])

print(f"✅ Whisper tamamlandı! ({len(transcript_segments)} segment)")
print(transcript_with_ts)

del model
torch.cuda.empty_cache()


✅ Ses çıkarıldı
Whisper yükleniyor...
Transkript alınıyor...
✅ Whisper tamamlandı! (430 segment)
[0:00] Welcome back everybody and this is my Black Friday special for 2025.
[0:04] I did one of these two years ago, I'm not sure why I didn't do it last year,
[0:06] but I'm back for 2025 Black Friday.
[0:09] I've got 39 products for you that meet three criteria.
[0:12] Number one, I used and or reviewed it.
[0:14] Number two, I liked it.
[0:15] And number three, it's actually on sale.
[0:17] Not one of these fake sales where it's the same price as always
[0:20] and they say it's on sale or they mark it up to mark it back down.
[0:23] So these 39 products about all three of these criteria,
[0:26] these are first-hand recommendations.
[0:27] Now for the three of you who remember my 2023 Black Friday special,
[0:31] you might recall that I used two browser extensions
[0:33] of Microsoft Shopping and Keepa.
[0:35] And I'm not using Microsoft Shopping this year for a couple reasons.
[0:38] Fir

In [10]:
print(transcript_with_ts)

[0:00] Welcome back everybody and this is my Black Friday special for 2025.
[0:04] I did one of these two years ago, I'm not sure why I didn't do it last year,
[0:06] but I'm back for 2025 Black Friday.
[0:09] I've got 39 products for you that meet three criteria.
[0:12] Number one, I used and or reviewed it.
[0:14] Number two, I liked it.
[0:15] And number three, it's actually on sale.
[0:17] Not one of these fake sales where it's the same price as always
[0:20] and they say it's on sale or they mark it up to mark it back down.
[0:23] So these 39 products about all three of these criteria,
[0:26] these are first-hand recommendations.
[0:27] Now for the three of you who remember my 2023 Black Friday special,
[0:31] you might recall that I used two browser extensions
[0:33] of Microsoft Shopping and Keepa.
[0:35] And I'm not using Microsoft Shopping this year for a couple reasons.
[0:38] First of all, I don't think it's as accurate as Keepa.
[0:40] Number two, it goes back too far in ti

## 3. LLM ile Ürün Çıkar (found/lost)

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Qwen2 7B - tamamen açık, çok iyi performans
MODEL_ID = "Qwen/Qwen2-7B-Instruct"

print("Qwen2 7B yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
llm_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)
print("✅ Qwen2 yüklendi!")

Qwen2 7B yükleniyor...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Qwen2 yüklendi!


In [17]:
import gc; gc.collect()
torch.cuda.empty_cache()

In [36]:
import json
import time
from tqdm.notebook import tqdm

def ask_llama(prompt, max_tokens=1000):
    messages = [
        {"role": "system", "content": "Sen bir ürün tespit asistanısın. JSON formatında cevap ver."},
        {"role": "user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(llm_model.device)
    outputs = llm_model.generate(input_ids, max_new_tokens=max_tokens, do_sample=True, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

def parse_json(text):
    """LLM çıktısından JSON parse et."""
    clean = text
    if "```json" in clean:
        clean = clean.split("```json")[1].split("```")[0]
    elif "```" in clean:
        clean = clean.split("```")[1].split("```")[0]
    start = clean.find("{")
    end = clean.rfind("}") + 1
    if start != -1 and end > start:
        clean = clean[start:end]
    return json.loads(clean)

def extract_products_from_chunk(chunk_text, chunk_num):
    """Tek bir chunk'tan ürün çıkar."""
    prompt = f"""

Bu bir YouTube videosunun transkript parçası. Videoda bahsedilen ürünleri bul.

== FOUND (SADECE marka/model varsa) ==
FOUND'a koymak için MARKA ŞART:
✓ "Logitech G Pro X" → FOUND (marka + model var)
✓ "Nintendo Switch" → FOUND (marka + ürün adı var)
✓ "iPad 8th generation" → FOUND (marka + nesil var)
✓ "Funko Pop Yoda" → FOUND (marka + karakter var)

== LOST (marka/model YOK) ==
Marka yoksa LOST'a koy, istisnasız:
✗ "gaming mouse" → LOST (hangi marka?)
✗ "tripod" → LOST (hangi tripod?)
✗ "headset" → LOST (marka yok)
✗ "wireless earbuds" → LOST (model yok)
✗ "stress ball" → LOST (marka yok)
✗ "water bottle" → LOST (marka yok)

== HİÇ EKLEME ==
- Fiyatlar: $100, "150 dollar item"
- Yerler: Japan, Tokyo, Amazon
- Kutular: box, mystery box, returns box
- Soyut kavramlar: disadvantage, stumbling block
- Servisler: YouTube, Underdog

KURAL: Marka yoksa → LOST. İstisna yok.

JSON:
{{"found": [{{"name": "...", "category": "...", "timestamp": "..."}}], "lost": [...]}}

Transkript:
---
{chunk}
---

SADECE JSON dön."""

    response = ask_llama(prompt, max_tokens=800)
    try:
        data = parse_json(response)
        found = data.get("found", [])
        lost = data.get("lost", [])
        return found, lost
    except:
        return [], []

# Transcript'i chunk'lara böl
CHUNK_SIZE = 3000  # karakter
chunks = []
for i in range(0, len(transcript_with_ts), CHUNK_SIZE):
    chunks.append(transcript_with_ts[i:i+CHUNK_SIZE])

print(f"Transcript {len(chunks)} chunk'a bölündü ({CHUNK_SIZE} karakter/chunk)")
print("="*50)

# Her chunk'ı işle
all_found = []
all_lost = []
start = time.time()

for i, chunk in enumerate(tqdm(chunks, desc="Ürünler çıkarılıyor"), 1):
    found, lost = extract_products_from_chunk(chunk, i)
    all_found.extend(found)
    all_lost.extend(lost)
    # Bellek temizle
    torch.cuda.empty_cache()

# Duplicate temizle (isme göre)
seen_names = set()
unique_found = []
for p in all_found:
    name = p.get("name", "").lower()
    if name not in seen_names:
        seen_names.add(name)
        unique_found.append(p)

unique_lost = []
for p in all_lost:
    name = p.get("name", "").lower()
    if name not in seen_names:
        seen_names.add(name)
        unique_lost.append(p)

print("="*50)
print(f"✅ Tamamlandı! ({time.time()-start:.1f} sn)")
print(f"FOUND: {len(unique_found)} ürün")
print(f"LOST: {len(unique_lost)} ürün")
print("="*50)

for p in unique_found:
    print(f"  🟢 [{p.get('timestamp', '?')}] {p.get('name')} ({p.get('category', '?')})")

for p in unique_lost:
    print(f"  🟡 [{p.get('timestamp', '?')}] {p.get('name')} ({p.get('category', '?')})")


Transcript 18 chunk'a bölündü (3000 karakter/chunk)


Ürünler çıkarılıyor:   0%|          | 0/18 [00:00<?, ?it/s]

✅ Tamamlandı! (427.9 sn)
FOUND: 122 ürün
LOST: 48 ürün
  🟢 [[0:00]] Amazon returns mystery boxes (Unknown)
  🟢 [[0:36]] 10 Amazon returns mystery boxes (Unknown)
  🟢 [[0:16]] Amazon returns (Unknown)
  🟢 [[0:14]] ancient dinosaur fossils (Unknown)
  🟢 [[0:11]] insane viral gadgets (Unknown)
  🟢 [[0:14]] actual house (Unknown)
  🟢 [[0:26]] mystery boxes (Unknown)
  🟢 [[0:46]] price (Unknown)
  🟢 [[0:48]] value (Unknown)
  🟢 [[0:50]] extreme punishment (Unknown)
  🟢 [[1:28]] Bingo spinner (Unknown)
  🟢 [[1:28]] disadvantage (Unknown)
  🟢 [[1:28]] no disadvantage (Unknown)
  🟢 [[1:19]] FaZe rug (Unknown)
  🟢 [[1:22]] Twist (Unknown)
  🟢 [[1:49]] lottery (Unknown)
  🟢 [[1:51]] paper (Unknown)
  🟢 [[2:42]] Advent calendar (holiday)
  🟢 [[3:00]] Drone (technology)
  🟢 [[3:28]] Paint (art supplies)
  🟢 [[3:41]] Wig (beauty)
  🟢 [[4:03]] Lens kit (photography)
  🟢 [[4:23]] Food truck (food service)
  🟢 [[4:47]] Impact socket adapter set (hardware)
  🟢 [[4:50]] Shoes (footwear)
  🟢 [[5:21]] Ant

In [ ]:
# JSON parse et
def parse_llm_json(text):
    clean = text
    if "```json" in clean:
        clean = clean.split("```json")[1].split("```")[0]
    elif "```" in clean:
        clean = clean.split("```")[1].split("```")[0]
    start_idx = clean.find("{")
    end_idx = clean.rfind("}") + 1
    if start_idx != -1 and end_idx > start_idx:
        clean = clean[start_idx:end_idx]
    return json.loads(clean)

try:
    data = parse_llm_json(llm_response)
    found = data.get("found", [])
    lost = data.get("lost", [])

    print(f"✅ FOUND ({len(found)} ürün):")
    for p in found:
        print(f"   🟢 {p['name']} ({p.get('category', '?')}) @ {p.get('timestamp', '?')}s")

    print(f"\n⚠️ LOST ({len(lost)} ürün) - Vision ile çözülecek:")
    for p in lost:
        print(f"   🟡 {p['name']} ({p.get('category', '?')}) @ {p.get('timestamp', '?')}s")
except Exception as e:
    print(f"❌ Parse hatası: {e}")
    found, lost = [], []

# Llama'yı temizle
del llm_model, tokenizer
torch.cuda.empty_cache()
print("\n✅ Llama bellekten silindi.")

In [ ]:
<cell_type>markdown</cell_type>---
## 4. Lost Ürünleri Vision ile Çöz

Video sadece lost varsa indirilir.

In [ ]:
if not lost:
    print("✅ Lost yok, Vision'a gerek yok!")
    vision_model = None
else:
    # Video indir (henüz indirilmediyse)
    if not local_video_path:
        print("Lost var, video indiriliyor...")
        local_video_path = download_video(VIDEO_URL, VIDEO_ID)

    if local_video_path:
        # LLaVA yükle
        from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
        from PIL import Image as PILImage

        print("LLaVA yükleniyor...")
        VISION_MODEL = "llava-hf/llava-v1.6-mistral-7b-hf"
        vision_processor = LlavaNextProcessor.from_pretrained(VISION_MODEL)
        vision_model = LlavaNextForConditionalGeneration.from_pretrained(
            VISION_MODEL,
            torch_dtype=torch.float16,
            device_map="auto",
            load_in_4bit=True,
        )
        print("✅ LLaVA yüklendi!")
    else:
        print("❌ Video indirilemedi, Vision atlanıyor")
        vision_model = None

In [ ]:
def extract_frame(video_path, timestamp_sec):
    """Video'dan belirli saniyede frame çıkar."""
    frame_path = f"{WORK_DIR}/frame_{timestamp_sec}.jpg"
    !ffmpeg -ss {timestamp_sec} -i "{video_path}" -vframes 1 -q:v 2 "{frame_path}" -y -loglevel error
    return frame_path if os.path.exists(frame_path) else None

def analyze_frame(image_path, product_hint):
    """LLaVA ile frame analiz et."""
    image = PILImage.open(image_path)
    prompt = f"Bu frame'de '{product_hint}' olarak bahsedilen ürünü tanıyabilir misin? Marka ve model adını söyle. Tanıyamıyorsan 'UNKNOWN' de."

    conversation = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": prompt}]}]
    formatted = vision_processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = vision_processor(images=image, text=formatted, return_tensors="pt").to(vision_model.device)
    output = vision_model.generate(**inputs, max_new_tokens=100, do_sample=False)
    result = vision_processor.decode(output[0], skip_special_tokens=True)
    if "[/INST]" in result:
        result = result.split("[/INST]")[-1].strip()
    return result

# Lost ürünleri çöz
if lost and vision_model:
    print(f"\n{len(lost)} lost ürün Vision ile analiz ediliyor...\n")
    resolved = []

    for product in lost:
        ts = product.get("timestamp", 0)
        name = product.get("name", "ürün")
        print(f"🔍 '{name}' @ {ts}s ...", end=" ")

        frame_path = extract_frame(local_video_path, ts)
        if frame_path:
            guess = analyze_frame(frame_path, name)
            print(f"→ {guess[:50]}")

            if guess and "UNKNOWN" not in guess.upper():
                product["name"] = guess.split("\n")[0][:100]  # İlk satır, max 100 char
                product["resolved_by"] = "vision"
                found.append(product)
                resolved.append(product)

    # Resolved olanları lost'tan çıkar
    for r in resolved:
        if r in lost:
            lost.remove(r)

    print(f"\n✅ {len(resolved)} ürün Vision ile çözüldü!")

In [ ]:
<cell_type>markdown</cell_type>---
## 5. Sonuçlar

In [ ]:
print("="*60)
print("SONUÇLAR")
print("="*60)

print(f"\n✅ FOUND ({len(found)} ürün):")
for i, p in enumerate(found, 1):
    resolved = " [Vision]" if p.get("resolved_by") == "vision" else ""
    print(f"   {i}. {p['name']} ({p.get('category', '?')}){resolved}")

print(f"\n❌ LOST ({len(lost)} ürün) - Tanımlanamadı:")
for i, p in enumerate(lost, 1):
    print(f"   {i}. {p['name']} ({p.get('category', '?')})")

print("\n" + "="*60)
print(f"Video indirildi mi: {'Evet' if local_video_path else 'Hayır'}")
print(f"Transcript kaynağı: {'YouTube' if not local_video_path else 'Whisper'}")
print("="*60)

---
## 6. LLaVA ile Frame Analizi

LLaVA - Open source vision-language model.

In [ ]:
def analyze_frame_with_llava(image_path, prompt):
    """LLaVA ile frame analiz et."""
    image = PILImage.open(image_path)

    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt},
            ],
        },
    ]

    formatted = vision_processor.apply_chat_template(
        conversation, add_generation_prompt=True
    )

    inputs = vision_processor(
        images=image,
        text=formatted,
        return_tensors="pt"
    ).to(vision_model.device)

    output = vision_model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
    )

    result = vision_processor.decode(output[0], skip_special_tokens=True)
    # Sadece cevabı al
    if "[/INST]" in result:
        result = result.split("[/INST]")[-1].strip()
    return result

# Test
if frames:
    print("Test frame analizi...")
    display(Image(filename=frames[0], width=300))

    test_result = analyze_frame_with_llava(
        frames[0],
        "What products or devices do you see in this image? List them briefly."
    )
    print(f"\nSonuç: {test_result}")

---
## 7. Sonuçları Birleştir

---
## 8. Performans ve Maliyet Özeti

---
## Notlar ve Değerlendirme

### Kalite Değerlendirmesi

| Model | Kalite (/10) | Notlar |
|-------|--------------|--------|
| Whisper | /10 | |
| Llama 3.1 | /10 | |
| LLaVA | /10 | |

### Bulunan Ürünler Doğru mu?
- [ ] Evet, çoğu doğru
- [ ] Yarı yarıya
- [ ] Çoğu yanlış

### Kaçan Ürünler:
- ...

### Yanlış Tespitler:
- ...

### API vs Open Source:
- [ ] Open source yeterli
- [ ] API daha iyi ama open source kabul edilebilir
- [ ] API şart, open source yetersiz

### Sonraki Adımlar:
- ...